# Aprendizado de Máquina - TP1
# Classificação de Exoplanetas

O trabalho tem como objetivo a prática de conceitos aprendidos na disciplina, adquirindo experiência no uso de métodos de classificação, na avaliação de modelos, e na interpretação e apresentação de resultados de experimentos. Para isso, utilizaremos e compararemos diferentes métodos para solucionar um problema de classificação binária.  

O problema abordado é o da classificação de exoplanetas identificados pela sonda espacial *Kepler* entre **confirmados** e **falsos positivos**. Os possíveis exoplanetas são chamados **Kepler Object of Interest (KOI)**, e cada observação do conjunto de dados corresponde a um KOI e suas características estimadas.

Serão explorados os seguintes métodos de classificação:
- [x] **Naive Bayes**
    - [x] Apenas um experimento, para servir de baseline.
- [x] **Decision Tree**
    - [x] Variação de altura máxima da árvore, incluindo ilimitada
    - [x] Visualização gráfica dos resultados
- [ ] **SVM**
    - [ ] Avaliação dos Kernels
        - [ ] Linear
        - [ ] Sigmoid
        - [ ] Polinomial
        - [ ] RBF
- [ ] **k-NN**
    - [ ] Variação do número de vizinhos *k*
    - [ ] Visualização gráfica dos resultados
- [ ] **Random Forest**
    - [ ] Variação do número de árvores
    - [ ] Visualização gráfica dos resultados
- [ ] **Gradient Tree Boosting**
    - [ ] Variação do número de iterações
    - [ ] Visualização gráfica dos resultados

Os métodos estão disponíveis no módulo `scikit-learn`. Iremos utilizar também os módulos `numpy`, `matplotlib.pyplot` e `pandas` para operações matemáticas, geração de gráficos, e manipulação do conjunto de dados, respectivamente.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Definimos um _seed_ para que os resultados do _notebook_ sejam reproduzíveis:

In [3]:
seed = 7

# Verificação e Tratamento de Dados

In [4]:
koi = pd.read_csv("koi_data.csv")
koi = koi.set_index("kepoi_name")
koi.isnull().sum()

koi_disposition     0
koi_period          0
koi_impact          0
koi_duration        0
koi_depth           0
koi_ror             0
koi_srho            0
koi_prad            0
koi_sma             0
koi_incl            0
koi_teq             0
koi_insol           0
koi_dor             0
koi_max_sngle_ev    0
koi_max_mult_ev     0
koi_model_snr       0
koi_steff           0
koi_slogg           0
koi_smet            0
koi_srad            0
koi_smass           0
koi_kepmag          0
koi_gmag            0
koi_rmag            0
koi_imag            0
koi_zmag            0
koi_jmag            0
koi_hmag            0
koi_kmag            0
koi_fwm_stat_sig    0
koi_fwm_sra         0
koi_fwm_sdec        0
koi_fwm_srao        0
koi_fwm_sdeco       0
koi_fwm_prao        0
koi_fwm_pdeco       0
koi_dicco_mra       0
koi_dicco_mdec      0
koi_dicco_msky      0
koi_dikco_mra       0
koi_dikco_mdec      0
koi_dikco_msky      0
dtype: int64

Não há dados faltantes no conjunto de dados.

In [5]:
target = koi["koi_disposition"] # Classificação de interesse
koi = koi.drop("koi_disposition", axis=1) # Essa separação facilita o uso dos métodos em sklearn
target = target.replace({"CONFIRMED": 1, "FALSE POSITIVE": 0}) # Essa transformação facilita o uso do módulo sklearn.metrics
target.index = koi.index
target.describe()

count    5202.000000
mean        0.404460
std         0.490834
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: koi_disposition, dtype: float64

Temos duas classes de interesse, e pela proporção observada de falsos positivos (3098/5202), não há sub-representação relevante.

In [6]:
koi.describe()

,koi_period,koi_impact,koi_duration,koi_depth,koi_ror,koi_srho,koi_prad,koi_sma,koi_incl,koi_teq,...,koi_fwm_srao,koi_fwm_sdeco,koi_fwm_prao,koi_fwm_pdeco,koi_dicco_mra,koi_dicco_mdec,koi_dicco_msky,koi_dikco_mra,koi_dikco_mdec,koi_dikco_msky
count,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,...,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000
mean,37.032237,0.717106,5.607025,21340.318993,0.235205,3.415370,112.230798,0.158146,81.181413,1143.721069,...,-0.355681,-0.805629,-0.000263,0.000439,-0.049743,-0.087413,1.930251,-0.038402,-0.098738,1.920226
std,88.417985,2.628207,6.962634,66989.808550,2.586213,25.131368,3699.799318,0.241792,16.308839,775.788868,...,10.978677,14.741473,0.065707,0.077519,2.465670,2.746534,3.147553,2.465094,2.734732,3.142764
min,0.306940,0.000000,0.104600,0.800000,0.001290,0.000040,0.080000,0.007200,2.290000,92.000000,...,-275.600000,-397.620000,-4.000000,-0.800000,-21.500000,-75.900000,0.000000,-23.600000,-76.600000,0.000000
25%,2.213962,0.226000,2.500250,176.800000,0.013058,0.176092,1.460000,0.033000,81.930000,615.250000,...,-0.500000,-0.570000,-0.000240,-0.000240,-0.270000,-0.291500,0.128250,-0.265250,-0.320000,0.180000
50%,7.386755,0.610000,3.805500,495.950000,0.024185,0.748045,2.600000,0.073650,87.890000,948.000000,...,0.000000,-0.030000,0.000000,0.000000,0.000000,0.000000,0.460000,-0.007000,-0.018000,0.453000
75%,23.448117,0.923750,6.000750,2120.525000,0.171260,2.267063,21.645000,0.158200,89.520000,1482.000000,...,0.500000,0.450000,0.000260,0.000280,0.230000,0.230000,2.570000,0.226250,0.250000,2.420000
max,1071.232620,100.806000,138.540000,864260.000000,99.870650,918.752390,200346.000000,2.034500,90.000000,9791.000000,...,97.780000,98.780000,1.190000,5.000000,45.680000,27.500000,88.600000,46.570000,31.200000,89.600000


**Temos muita variação entre as grandezas dos atributos.** (e.g: `koi_depth` e `koi_fwm_pdeco`)  
**Deve ser interessante normalizar os dados para melhor funcionamento de alguns modelos.**

In [7]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
koi_minmax = min_max_scaler.fit_transform(koi.values)
koi_minmax = pd.DataFrame(koi_minmax,columns=koi.columns, index=koi.index)
koi_minmax.head()

,koi_period,koi_impact,koi_duration,koi_depth,koi_ror,koi_srho,koi_prad,koi_sma,koi_incl,koi_teq,...,koi_fwm_srao,koi_fwm_sdeco,koi_fwm_prao,koi_fwm_pdeco,koi_dicco_mra,koi_dicco_mdec,koi_dicco_msky,koi_dikco_mra,koi_dikco_mdec,koi_dikco_msky
kepoi_name,,,,,,,,,,,,,,,,,,,,,
K00752.01,0.008573,0.001448,0.020608,0.000712,0.000211,0.003492,0.000011,0.038524,0.996124,0.072275,...,0.739274,0.802901,0.770674,0.137836,0.319887,0.735977,0.002257,0.337466,0.713451,0.003571
K00752.02,0.050528,0.005813,0.031801,0.001011,0.000267,0.003291,0.000014,0.131308,0.995097,0.036189,...,0.736435,0.803485,0.770840,0.137750,0.325841,0.734043,0.004402,0.343309,0.711688,0.005580
K00754.01,0.001335,0.012658,0.016627,0.009347,0.003866,0.000240,0.000167,0.009619,0.738798,0.134344,...,0.737825,0.801011,0.771295,0.137686,0.316329,0.735464,0.003262,0.332664,0.711494,0.003080
K00755.01,0.002072,0.006954,0.011196,0.000697,0.000228,0.002162,0.000013,0.014897,0.947668,0.135478,...,0.738095,0.801471,0.770728,0.137919,0.320482,0.733172,0.001129,0.337324,0.710761,0.000781
K00114.01,0.006588,0.011597,0.035521,0.000269,0.001823,0.000005,0.000195,0.036896,0.668453,0.128879,...,0.702100,0.849537,0.771297,0.136974,0.252962,0.808607,0.100790,0.271669,0.782124,0.099866


# Validação Cruzada K-Fold

Para evitar [*overfitting*](https://en.wikipedia.org/wiki/Overfitting) em um classificador, dividimos o conjunto de dados em subconjuntos de **treino** e **teste**. Essa técnica é chamada de [**validação cruzada**.](https://en.wikipedia.org/wiki/Cross-validation_(statistics))
- **Treino**: dados a partir dos quais serão estimados os parâmetros do classificador.
- **Teste**: dados usados para avaliar a acurácia do classificador para dados desconhecidos (i.e., avaliar sua generalização).

Existem diversas estratégias de validação cruzada. Neste trabalho usaremos a validação **k-fold**, a fim de estimar a qualidade de um modelo com boa confiabilidade. Ela se dá pelos seguintes passos:
- O conjunto de dados é dividido em *k* partes de tamanhos iguais;
- O modelo é treinado _k_ vezes, em cada uma delas usando um dos subconjuntos como conjunto de teste e a união dos *k-1* subconjuntos restantes como conjunto de treinamento;
    - Para cada iteração, o modelo é avaliado.
- Por fim, a qualidade do modelo é sumarizada a partir dos resultados obtidos.

Para tanto, aproveitaremos a implementação `KFold`, presente no módulo `sklearn`. Se uma das classes de exoplanetas estivesse sub-representada, seria interessante formar subconjuntos estratificados com o `StratifiedKFold`.

In [8]:
from sklearn.model_selection import KFold
kfold = KFold(5, shuffle=True, random_state=seed) # 5 e 10 são valores comumente escolhidos para k

Exemplo de funcionamento do `KFold`:

In [9]:
for train, test in kfold.split(koi[:50], target[:50]):
    print("TRAIN:", train, "TEST:", test)

TRAIN: [ 0  2  3  4  5  6  7  8  9 11 12 14 16 17 18 19 21 23 24 25 26 28 29 31
 32 33 34 35 37 38 39 40 41 43 44 45 46 47 48 49] TEST: [ 1 10 13 15 20 22 27 30 36 42]
TRAIN: [ 0  1  3  4  5  6  7  8 10 11 12 13 14 15 16 19 20 21 22 23 24 25 26 27
 28 30 31 33 36 37 38 39 40 42 43 44 45 47 48 49] TEST: [ 2  9 17 18 29 32 34 35 41 46]
TRAIN: [ 0  1  2  3  4  7  8  9 10 11 13 14 15 17 18 19 20 22 23 25 26 27 28 29
 30 32 34 35 36 37 39 40 41 42 43 44 46 47 48 49] TEST: [ 5  6 12 16 21 24 31 33 38 45]
TRAIN: [ 1  2  3  4  5  6  9 10 12 13 14 15 16 17 18 19 20 21 22 23 24 25 27 28
 29 30 31 32 33 34 35 36 38 39 41 42 44 45 46 47] TEST: [ 0  7  8 11 26 37 40 43 48 49]
TRAIN: [ 0  1  2  5  6  7  8  9 10 11 12 13 15 16 17 18 20 21 22 24 26 27 29 30
 31 32 33 34 35 36 37 38 40 41 42 43 45 46 48 49] TEST: [ 3  4 14 19 23 25 28 39 44 47]


# Avaliação dos experimentos

Cada experimento realizado será avaliado pelas seguintes métricas:
- Acurácia
- Revocação
- Precisão 
- Área abaixo da curva ROC (ROC AUC)

Para melhor organizar e facilitar o acesso os resultados, estes serão guardados em um `DataFrame`.

In [10]:
results = pd.DataFrame(np.zeros(shape=(12,4)),columns=['Acurácia', 'Precisão', 'Revocação', 'ROC AUC'])
indexes = [['Naive Bayes', 'Decision Tree', 'SVM', 'k-NN', 'Random Forest', 'Gradient Tree Boosting'], ['Média', 'Desvio Padrão']]
results.index = pd.MultiIndex.from_product(indexes, names=['Método', 'Medida'])

Uma função para empacotar as avaliações de um modelo será bastante útil:

In [11]:
from sklearn import metrics

def evaluate_prediction(y, y_pred):
    return [
        metrics.accuracy_score(y, y_pred),
        metrics.precision_score(y, y_pred),
        metrics.recall_score(y, y_pred),
        metrics.roc_auc_score(y, y_pred)
    ]

# Naive Bayes 

O Naive Bayes é um método que parte da suposição de que os atributos ($X$) das observações são condicionalmente independentes entre si, dado o valor da classe de interesse ($y$, ou `target`). Partindo dessa hipótese, a probabilidade de ocorrência de cada possível valor de `y` para uma observação do nosso conjunto de dados é facilmente calculado usando o teorema de Bayes.  

A classificação de uma observação é então dada por:
$$\begin{align}\begin{aligned}\\\hat{y} = \arg\max_y P(y) \prod_{i=1}^{n} P(x_i \mid y),\end{aligned}\end{align}$$

Para o cálculo de $P(x_i \mid y)$, assumiremos que as distribuições de probabilidade das _features_ seguem uma normal. Por ser uma distribuição frequentemente observada, parece uma suposição razoável. O objeto `GaussianNB` realiza a parametrização do Naive Bayes dessa forma. 

Faremos apenas um experimento, que servirá como uma primeira tentativa de solução do problema, e base de comparação para os outros métodos.

In [12]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb_results = []
for train, test in kfold.split(koi, target):
    y_pred = gnb.fit(koi.iloc[train], target[train]).predict(koi.iloc[test])
    gnb_results.append(evaluate_prediction(target[test], y_pred))
gnb_results = np.array(gnb_results)
results.loc["Naive Bayes"] =  [gnb_results.mean(axis=0), gnb_results.std(axis=0)]
results.loc["Naive Bayes"]

,Acurácia,Precisão,Revocação,ROC AUC
Medida,,,,
Média,0.797388,0.672511,0.973880,0.825687
Desvio Padrão,0.008703,0.011312,0.008182,0.006910


Embora o valor de revocação seja alto, a precisão do modelo é péssima; isto é, a classificação está muito branda, e o modelo acerta muito na classe de interesse, enquanto também produz muitos falsos positivos.  

O cálculo de $P(y \mid X)$ está favorecendo $P(y = 1 \mid X)$ mais frequentemente do que deveria. Pode ser que classe a classe de exoplanetas $y = 0$ tenha _outliers_ para algumas features, enviesando as médias $\mu_0$ e consequentemente fazendo com que $\prod_{i=1}^{n} P(x_i \mid y = 1)$ seja mais frequentemente maior que $\prod_{i=1}^{n} P(x_i \mid y = 0)$, por se tratarem de distribuições gaussianas.

Além disso, a suposição ingênua de que os atributos de um KOI são condicionalmente independentes não é condizente com o problema real. Por exemplo, as dimensões de um corpo espacial têm influência sobre o seu trajeto no espaço.

---

# Decision Tree

Árvores de decisão podem ser usadas tanto para classificação (o problema em questão) quanto para regressão. A ideia é prever a classe de um elemento por meio do reconhecimento de regras simples de decisão, deduzidas pelos dados de treino. Em outras palavras, o algoritmo classifica o elemento de entrada a partir de "perguntas" sobre seus atributos, as quais o classificam a partir da classe de observações do conjunto de treino que deram as mesmas respostas. 

Esse fluxo é representado por uma árvore binária, em que cada nó tem um conjunto de indivíduos e uma proposição lógica (a "pergunta"), cujo valor é usado para obter dois subconjuntos, os quais serão atribuídos aos nós filhos. Estes, de mesma forma, terão seus dados divididos por outra proposição, e assim por diante, até que subconjuntos puros sejam obtidos, ou a profundidade máxima da árvore seja atingida. 

Uma entrada é classificada de acordo com o nó-folha ao qual suas respostas às perguntas da árvore a direcionarem. Se o nó-folha não for associado a um subconjunto homogêneo, a classe associada será a de maior frequência.

Vamos realizar experimentos variando a altura máxima permitida da árvore, a fim de analisar o impacto desse hiperparâmetro. O módulo `sklearn.tree` contém a implementação de uma árvore de decisão classificadora (`DecisionTreeClassifier`).

O algoritmo será executado escolhendo as proposições que melhor dividem o conjunto de dados de um nó, usando a medida de impureza de Gini. Cada divisão será feita escolhendo a proposição que minimiza essa medida nas suas divisões, de forma gulosa. Também é possível selecionar proposições de forma estocástica, a fim de evitar _overfitting_ do modelo.

Para começar, vamos experimentar usar uma árvore de tamanho máximo ilimitado. Isto é, a árvore cresce em profundidade o quanto for necessário para que os nós-folha tenham conjuntos puros. A árvore resultante terá acurácia perfeita para os dados de treino, mas deve sofrer de _overfitting_; por ser demasiadamente complexa para classificar o conjunto de treinamento perfeitamente, provavelmente não é um modelo muito generalizável. 

### Árvore de decisão com profundidade máxima ilimitada

In [13]:
from sklearn import tree

dt_maxdepth = tree.DecisionTreeClassifier()
maxdepth_results = []
trees = []
for train, test in kfold.split(koi, target):
    dt_maxdepth = dt_maxdepth.fit(koi.iloc[train], target.iloc[train])
    y_pred = dt_maxdepth.predict(koi.iloc[test])
    maxdepth_results.append(evaluate_prediction(target[test], y_pred))
maxdepth_results = np.array(maxdepth_results)
results.loc["Decision Tree"] = [maxdepth_results.mean(axis=0), maxdepth_results.std(axis=0)]
results.loc["Decision Tree"]

,Acurácia,Precisão,Revocação,ROC AUC
Medida,,,,
Média,0.950403,0.942732,0.934340,0.947783
Desvio Padrão,0.002408,0.010804,0.008594,0.001971


Em média, a árvore de decisão de profundidade ilimitada, embora apresente menor valor de revocação, já supera muito o Naive Bayes em todas as outras métricas.

A profundidade máxima da árvore foi de:

In [14]:
dt_maxdepth.tree_.max_depth

28

### Árvore de decisão com profundidade máxima 3

In [15]:
dt_depth3 = tree.DecisionTreeClassifier(max_depth=3)
depth3_results = []
trees = []
for train, test in kfold.split(koi, target):
    dt_depth3 = dt_depth3.fit(koi.iloc[train], target.iloc[train])
    y_pred = dt_depth3.predict(koi.iloc[test])
    depth3_results.append(evaluate_prediction(target[test], y_pred))
depth3_results = np.array(depth3_results)
results.loc["Decision Tree"] = [depth3_results.mean(axis=0), depth3_results.std(axis=0)]
results.loc["Decision Tree"]

,Acurácia,Precisão,Revocação,ROC AUC
Medida,,,,
Média,0.929258,0.890792,0.941007,0.931110
Desvio Padrão,0.006098,0.017847,0.010112,0.004443


Em média, a qualidade do nosso modelo, em geral, piorou. A árvore precisa de maior profundidade para dividir bem as classes.

### Árvore de decisão com profundidade máxima 5

In [16]:
dt_depth5 = tree.DecisionTreeClassifier(max_depth=5)
depth5_results = []
trees = []
for train, test in kfold.split(koi, target):
    dt_depth5 = dt_depth5.fit(koi.iloc[train], target.iloc[train])
    y_pred = dt_depth5.predict(koi.iloc[test])
    depth5_results.append(evaluate_prediction(target[test], y_pred))
depth5_results = np.array(depth5_results)
results.loc["Decision Tree"] = [depth5_results.mean(axis=0), depth5_results.std(axis=0)]
results.loc["Decision Tree"]

,Acurácia,Precisão,Revocação,ROC AUC
Medida,,,,
Média,0.952902,0.928192,0.957573,0.953599
Desvio Padrão,0.004906,0.004384,0.014828,0.006393


Em média, conseguimos revocação melhor que a primeira árvore, com um pouco menos de precisão, mantendo a acurácia e ROC AUC semelhantes. Aumentar mais a altura da árvore pode gerar resultados melhores, embora deva-se manter em mente que à medida que esse valor aumenta, maior o risco de _overfitting_. 

### Árvore de decisão com profundidade máxima 10

In [17]:
dt_depth10 = tree.DecisionTreeClassifier(max_depth=10)
depth10_results = []
trees = []
for train, test in kfold.split(koi, target):
    dt_depth10 = dt_depth10.fit(koi.iloc[train], target.iloc[train])
    y_pred = dt_depth10.predict(koi.iloc[test])
    depth10_results.append(evaluate_prediction(target[test], y_pred))
depth10_results = np.array(depth10_results)
results.loc["Decision Tree"] = [depth10_results.mean(axis=0), depth10_results.std(axis=0)]
results.loc["Decision Tree"]

,Acurácia,Precisão,Revocação,ROC AUC
Medida,,,,
Média,0.953095,0.934634,0.950439,0.952617
Desvio Padrão,0.005879,0.009113,0.010180,0.006286


Não atingimos melhoria nos resultados, em média. Então, conclui-se que uma árvore de profundidade máxima 5 já é razoável para classificação do problema utilizando esse método. Para futura comparação com outros classificadores, vamos guardar o seu resultado por ter menos risco de _overfitting_ que as mais profundas.

In [18]:
results.loc["Decision Tree"] = [depth5_results.mean(axis=0), depth5_results.std(axis=0)]

Uma propriedade interessante das árvores de decisão é que o processo de classificação é uma "caixa branca", que ao contrário da "caixa preta" de outros métodos, podemos visualizar e entender facilmente.

No módulo `sklearn.tree` podemos exportar as árvores como representação _dot_, e usar o módulo `pydot` para salvá-las como imagens:

In [19]:
import pydot

def image_from_tree(dt, filename):
    dot_data = tree.export_graphviz(dt, feature_names=koi.columns,  
                     class_names=["FALSE POSITIVE", "CONFIRMED"],  
                     filled=True, rounded=True,
                     special_characters=True) 
    (graph, ) = pydot.graph_from_dot_data(dot_data)
    graph.write_png(filename) 
image_from_tree(dt_maxdepth, "dt_maxdepth.png")
image_from_tree(dt_depth3, "dt_depth3.png")
image_from_tree(dt_depth5, "dt_depth5.png")

Na representação gráfica, temos as perguntas, a impureza de Gini de cada subconjunto, o número de amostras no subconjunto, a quantidade de elementos por classe, e a classe associada ao nó. Além disso, a cor dos nós representa a sua homogeneidade.

![Árvore de Profundidade Ilimitada](dt_maxdepth.png)
É possível ver que muitas das folhas da árvore de máxima profundidade possuem uma ou poucas observações, e é esse o _overfitting_ do modelo; condições demasiadamente complexas para **perfeitamente** classificar, **especificamente**, o conjunto de treinamento.

![Árvore de Profundidade Máxima 3](dt_depth3.png)

![Árvore de Profundidade Máxima 5](dt_depth5.png)
Com a representação das árvores de profundidade 3 e 5 lado a lado, fica evidente a necessidade de expansão da árvore para aumentar a acurácia do modelo; várias ramificações após as folhas da primeira árvore alteram a saída do modelo.

---

# SVM

**Support-Vector Machines** (SVM) são modelos de aprendizado supervisionado, usados para classificação e análise de regressão. Diferentemente do Naive Bayes e das Árvores de Decisão, trata-se de um modelo não-probabilístico; isto é, um SVM apenas atribui uma entrada a uma categoria ou outra, sem obter distribuições de probabilidade (embora existam métodos para adapatar o SVM a esse contexto). Além disso, o SVM é um classificador binário (embora existam procedimentos que permitem estender o SVM para problemas multiclasse).

O funcionamento do SVM consiste em representar o conjunto de dados em um espaço geométrico em que cada atributo define uma dimensão, para, então, construir hiperplanos nesse espaço. Em um problema de classificação, estes hiperplanos servem como um separador entre os representantes de cada classe, mas podem ser usados também para regressão, entre outras tarefas. 

É comum que os conjuntos de interesse não sejam linearmente separáveis no espaço construído. Para contornar essa limitação do uso de hiperplanos, o espaço é mapeado para outro de muito mais dimensões, presumivelmente tornando as classes mais facilmente separáveis. Esse mapeamento é definido em termos de uma função **kernel**, escolhida de acordo com sua adequação ao problema. Em outras palavras, a _kernel_ define a curvatura, ou "desenho", do hiperplano.

Vamos realizar experimentos a fim de analisar a performance do SVM para o nosso problema de classificação de KOIs, avaliando a adequação das funções linear, sigmoide, polinomial e RBF como _kernel_. A classe `sklearn.SVM.SVC` implementa a classificação binária por SVM. Para melhorar a performance do SVM, utilizaremos dados **normalizados** _(a experimentação com valores absolutos levou tempo indeterminado, tornando-se inviável considerar a opção)_. Para o valor de **C**, hiperparâmetro que penaliza erros de classificação no treinamento do modelo, os melhores resultados encontrados foram com $C = 1000$, escolhendo hiperplanos com menor margem de distância dos pontos.

### SVM com kernel linear

In [42]:
from sklearn.svm import SVC
svm_linear = SVC(kernel='linear', C=1000)
svm_linear_results = []
for train, test in kfold.split(koi_minmax, target):
    svm_linear = svm_linear.fit(koi_minmax.iloc[train], target.iloc[train])
    y_pred = svm_linear.predict(koi_minmax.iloc[test])
    svm_linear_results.append(evaluate_prediction(target.iloc[test], y_pred))
svm_linear_results = np.array(svm_linear_results)
results.loc["SVM"] = [svm_linear_results.mean(axis=0), svm_linear_results.std(axis=0)]
results.loc["SVM"]

,Acurácia,Precisão,Revocação,ROC AUC
Medida,,,,
Média,0.948482,0.921266,0.954321,0.949385
Desvio Padrão,0.003858,0.009394,0.005315,0.003224


### SVM com kernel sigmoide

Para kernels não-lineares, o hiperparâmetro _gamma_ define o quanto o modelo tenta se adequar perfeitamente ao conjunto de treinamento. Valores altos de _gamma_ causam _overfitting_. Escolhemos `'scale'`, que na verdade é $1/n*\sigma^2$, sendo $n$ o número de _features_, e $\sigma^2$ a variância dos dados.

In [39]:
from sklearn.svm import SVC
svm_linear = SVC(kernel='sigmoid', gamma='scale')
svm_linear_results = []
for train, test in kfold.split(koi_minmax, target):
    svm_linear = svm_linear.fit(koi_minmax.iloc[train], target.iloc[train])
    y_pred = svm_linear.predict(koi_minmax.iloc[test])
    svm_linear_results.append(evaluate_prediction(target.iloc[test], y_pred))
svm_linear_results = np.array(svm_linear_results)
results.loc["SVM"] = [svm_linear_results.mean(axis=0), svm_linear_results.std(axis=0)]
results.loc["SVM"]

,Acurácia,Precisão,Revocação,ROC AUC
Medida,,,,
Média,0.811799,0.829110,0.672561,0.789467
Desvio Padrão,0.022724,0.021365,0.048328,0.026650


### SVM com kernel polinomial

In [38]:
from sklearn.svm import SVC
svm_linear = SVC(kernel='poly', degree=3, gamma='scale', C=1000)
svm_linear_results = []
for train, test in kfold.split(koi_minmax, target):
    svm_linear = svm_linear.fit(koi_minmax.iloc[train], target.iloc[train])
    y_pred = svm_linear.predict(koi_minmax.iloc[test])
    svm_linear_results.append(evaluate_prediction(target.iloc[test], y_pred))
svm_linear_results = np.array(svm_linear_results)
results.loc["SVM"] = [svm_linear_results.mean(axis=0), svm_linear_results.std(axis=0)]
results.loc["SVM"]

,Acurácia,Precisão,Revocação,ROC AUC
Medida,,,,
Média,0.943675,0.907902,0.958108,0.94595
Desvio Padrão,0.008691,0.014617,0.008110,0.00825


### SVM com kernel RBF

In [36]:
from sklearn.svm import SVC
svm_linear = SVC(kernel='rbf', gamma='scale', C=1000)
svm_linear_results = []
for train, test in kfold.split(koi_minmax, target):
    svm_linear = svm_linear.fit(koi_minmax.iloc[train], target.iloc[train])
    y_pred = svm_linear.predict(koi_minmax.iloc[test])
    svm_linear_results.append(evaluate_prediction(target.iloc[test], y_pred))
svm_linear_results = np.array(svm_linear_results)
results.loc["SVM"] = [svm_linear_results.mean(axis=0), svm_linear_results.std(axis=0)]
results.loc["SVM"]

,Acurácia,Precisão,Revocação,ROC AUC
Medida,,,,
Média,0.943098,0.907711,0.956655,0.945229
Desvio Padrão,0.008713,0.013816,0.010205,0.008581


Embora melhor que o Naive Bayes, o SVM não se saiu melhor do que as árvores de decisão. Como a acurácia resultante do experimento com _kernel_ linear foi, em média, maior do que as outras opções de _kernel_, o problema é, aparentemente, de natureza linear.